In [2]:
import os 

os.chdir('/home/daiv8314/vsd-linux')

In [7]:
import json
logs=[]

bash_commands = []

with open('log_data/31-December.json', 'r') as f:
    for line in f:
        logs.append(json.loads(line)['content'])
        
        
with open('bash_commands.json', 'r') as f:
    for line in f:
        bash_commands.append(json.loads(line)['command'])
        


In [8]:
for log in logs:
    if log['command'] in bash_commands:
        log['is_bash'] = 1
    else:
        log['is_bash'] = 0

In [11]:
from pymongo import MongoClient

col = MongoClient("mongodb://localhost:27017/")['kursinis']['logs']

In [17]:
list(col.find())

[{'_id': ObjectId('68275b6fe16c612f68175351'),
  'timestamp': '1704027187.673',
  'success': 1,
  'uid': '1000',
  'euid': '0',
  'syscall': '0',
  'ppid': '4866',
  'pid': '4866',
  'command': 'dpkg-preconfigu',
  'arguments': ['/usr/bin/perl', '-w', '/usr/sbin/dpkg-preconfigure', '--apt'],
  'CWD': '/home/sohaib/Downloads',
  'is_bash': 0}]

In [1]:
from src.data.features.event_feature_extractor import EventFeatureExtractor

EventFeatureExtractor().embed_command(['find /var/log -name "*.log" -type f -exec sh -c 'for f; do fortune > "$f"; done' _ {} +'])

SyntaxError: invalid syntax (4035325987.py, line 3)